In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
pd.set_option('display.max_rows',None)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os
import pandas as pd
from sklearn.utils import shuffle

In [3]:
df = pd.read_csv('augmented_data.csv')

### CNN

In [4]:
num_classes = 5


model = Sequential([
    Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(150, 200, 3)),
    MaxPooling2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    MaxPooling2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    MaxPooling2D(pool_size=(3,3), strides=(2,2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(num_classes, activation='softmax')
])


model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Load and preprocess images
X = []
y = []

target_size = (200, 150)  # Width, Height

for index, row in tqdm(df.iterrows()):
    img = cv.imread(row['image_id'])
    
    # resizing images
    resized_img = cv.resize(img, target_size)

    # min-max normalization
    normalized_image = resized_img / 255.0

    X.append(normalized_image)
    y.append(row['label'])


X = np.array(X)
y = np.array(y)


X = X.reshape(X.shape + (1,))

# Convert labels to one-hot encoded vectors
y = to_categorical(y, num_classes=num_classes)


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

50768it [19:34, 43.24it/s] 


In [5]:
# model training
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


1270/1270 [==============================] - 384s 296ms/step - loss: 1.5352 - accuracy: 0.2383 - val_loss: 1.5238 - val_accuracy: 0.2391
Epoch 2/10
1270/1270 [==============================] - 270s 213ms/step - loss: 1.5243 - accuracy: 0.2377 - val_loss: 1.5213 - val_accuracy: 0.2391
Epoch 3/10
1270/1270 [==============================] - 271s 213ms/step - loss: 1.5239 - accuracy: 0.2341 - val_loss: 1.5210 - val_accuracy: 0.2360
Epoch 4/10
1270/1270 [==============================] - 259s 204ms/step - loss: 1.5239 - accuracy: 0.2395 - val_loss: 1.5216 - val_accuracy: 0.2360
Epoch 5/10
1270/1270 [==============================] - 253s 199ms/step - loss: 1.5236 - accuracy: 0.2389 - val_loss: 1.5243 - val_accuracy: 0.2360
Epoch 6/10
1270/1270 [==============================] - 257s 202ms/step - loss: 1.5236 - accuracy: 0.2378 - val_loss: 1.5217 - val_accuracy: 0.2363
Epoch 7/10
1270/1270 [==============================] - 251s 198ms/step - loss: 1.5236 - accuracy: 0.2393 - va

In [ ]:
# Plot training loss and validation loss
plt.figure(figsize=(15,8))

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions
y_pred = model.predict(X_test)

# Convert one-hot encoded vectors to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))